## <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Google_Drive_icon_%282020%29.svg/538px-Google_Drive_icon_%282020%29.svg.png" height="40" align="left"/><font size=6>&nbsp;&nbsp;Google Drive</font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <h3>← Mount/Unmount Google Drive</h3>
# @markdown <p>This cell will mount/unmount Google Drive to /content/drive/</p></br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
# ================================================================ #

from IPython.display import clear_output
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

clear_output()

## Extract Album Artwork from Any Audio File

<img src="https://cdn.jsdelivr.net/gh/TheKVT/Artwork_Extractor/images/audio.png" alt="audio-image" width="auto" height="auto"/>

In [ ]:
#@markdown <h4><b>Convert your file to mp3 by changing it won't encode image in audio file</b></h4>

import os, sys, re

audio_file_path = "" #@param {type:"string"}


output_file_path = re.search("^[\/].+\/", audio_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", audio_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = audio_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw

!ffmpeg -i "$inputFile"  -b:a 320k -acodec libmp3lame  -vcodec copy "$outputPath"/"$fileName"converted."mp3"

In [ ]:
#@title Extract Album Art from MP3 Audio
File_Path = "" #@param {type:"string"}
Output_Path = "" #@param {type:"string"}

import os
from IPython.display import HTML,clear_output

!pip install eyeD3

clear_output() 

os.makedirs(Output_Path, exist_ok=True)

!eyeD3 --write-images "$Output_Path" "$File_Path"

clear_output()

display(HTML("<center><h2 style=\"color:#00b24c;\"> Album Art Successfully Extracted..!!</h2><br></center>"))

In [ ]:
#@title Extract Album Art from FLAC Audio
Flac_Path = "" #@param {type:"string"}
Output_Path = "" #@param {type:"string"}
 
import os 
import IPython
import ipywidgets as widgets
 
from IPython.display import HTML, clear_output
print("Downloading...") 
get_ipython().system_raw("rm -rf /content/sample_data/ && sudo apt update && sudo apt install software-properties-common")
get_ipython().system_raw("pip3 -q install mutagen")
 
# os.makedirs(Output_Path, exist_ok=True) 
%cd $Output_Path
clear_output()
 
from mutagen.flac import FLAC, Picture
song = Flac_Path
var = FLAC(song)
pics = var.pictures
print (pics)
for p in pics:
    if p.type == 3: #front cover
        display(HTML("<center><h2 style=\"font-family:Merriweather;color:#00b24c;\">Album Art Successfully Exported!</h2><br></center>")) 
        with open("cover.jpg", "wb") as f:
            f.write(p.data)

## Extract Album Artwork from [Apple Music](https://music.apple.com/)(iTunes) and [Bugs!](https://music.bugs.co.kr/)

<img src="https://cdn.jsdelivr.net/gh/TheKVT/Artwork_Extractor/images/Apple.png" alt="Appleaudio-image" width="auto" height="auto"/>

In [ ]:

##################################################
#
# Code Author: jamescarter2001
#
# Remastered by TheKVT
#
##################################################

# ============================= FORM ============================= #
#@title
Mode = "Apple" #@param ["Apple", "Bugs"]
# @markdown > Select mode (Apple Music / Bugs)
# @markdown ---
url = "" #@param {type:"string"}
Picture_Resoultion = "3000" #@param ["1000", "1500", "2000", "2500", "3000"]
country = "us" #@param {type:"string"}
# @markdown > `country` only needed for `Apple`
output_path = "" #@param {type:"string"}
variable_name = "" #@param {type:"string"}

# ================================================================ #

!pip install pycountry wget bs4

import requests
import json
import os
import pycountry
import wget
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.display import clear_output
os.makedirs(output_path, exist_ok=True)

clear_output()

############## Apple Music ############
if Mode == "Apple": 
   res = Picture_Resoultion

   region = pycountry.countries.get(alpha_2=country)

   language = "en_us"  # ja_jp

   link = url.split('/')[-1].split('?')[0]

   itunes_endpoint = f'https://itunes.apple.com/lookup?id={link}&country={region.alpha_2}&lang={language}'

   response = requests.get(itunes_endpoint)
   if response.status_code == 200: 
    file_name = response.json()["results"][0]["collectionName"]
    artwork_link = response.json()["results"][0]["artworkUrl100"]
    artwork_link = artwork_link.replace("100x100bb.jpg", f"{res}x{res}bb.jpg")
    url = artwork_link
    path = output_path
    wget.download(url, f'{path}/{file_name}-cover.jpg')
    print("Downloaded " + file_name + " Album ArtWork Sucessfully..!!")
    
## if you get "index error" means 'item not avilable in apple music of the country you haveentered'

############## Bugs ############

if Mode == "Bugs":
  myurl = url
  soup = BeautifulSoup(urlopen(myurl), "lxml")
  kvt = soup.title.get_text()
  name = kvt.replace(' - 벅스', '')
  resolution = Picture_Resoultion # change resolution in form
  artwork_base_url = f'https://image.bugsm.co.kr/album/images/{resolution}'

  album_url = url
  album_id = album_url.split('/')[-1].split('?')[0]
  id = album_id[0:-2]

  artwork_url = f'{artwork_base_url}/{id}/{album_id}.jpg'
  dl = artwork_url
  path = output_path

  wget.download(dl, f'{path}/{name}-cover.jpg')
  print("Downloaded " + name + " Album ArtWork Sucessfully..!!")

## Misc

In [ ]:
#@title ##Upload to your image to [transfer.sh](https://transfer.sh)
#@markdown In *file_name* add your custom name and default extension

file_path = "" #@param {type:"string"}
file_name = "Album_Cover(TheKVT).jpg" #@param {type:"string"}
max_downloads = "4" #@param {type:"string"}
store_days = "1" #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14"]

import os,re
os.environ['inputFile'] = file_path 
os.environ['maxDl'] = max_downloads
os.environ['daysStore'] = store_days
os.environ['fileName'] = file_name

mylink = !curl -H "Max-Downloads: "$maxDl"" -H "Max-Days: "$daysStore"" --upload-file "$inputFile" https://transfer.sh/"$fileName"
print('Here your image link:-  ' + str(mylink))

In [ ]:
#@title ## Get file Metadata from MediaInfo
path = "" #@param {type:"string"}
import os, uuid, re, IPython
from IPython.display import HTML
import ipywidgets as widgets
import time

def mediainfo():
  display(HTML("<br>"))
  display(HTML("<br>"))
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")
  
if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")
  
mediainfo()